In [1]:
import json
import os

from selenium import webdriver
from bs4 import BeautifulSoup

from tqdm.auto import tqdm
from multiprocess import Process, Manager, Lock

In [2]:
SRC = 'data/src'

os.makedirs(SRC, exist_ok=True)

In [3]:
chrome_options = webdriver.ChromeOptions()
chrome_prefs = {"profile.managed_default_content_settings.images": 2}
chrome_options.add_experimental_option("prefs", chrome_prefs)

## Выгружаем айдишники домов

In [4]:
LIMIT = 1000

ids = set()

In [5]:
with webdriver.Chrome(options=chrome_options) as browser:
    cur = 0
    batch = 10

    while cur < LIMIT:
        browser.get(f'https://наш.дом.рф/сервисы/api/kn/object?offset={cur}&limit={batch}&sortField=objId&objStatus=0')    
        soup = BeautifulSoup(browser.page_source, 'html.parser')
        
        try:
            q = json.loads(soup.find('pre').get_text())
        except:
            print(soup)
            break
        
        if not q:
            break

        ids |= set(elem['objId'] for elem in q['data']['list'])

        cur += batch


## Скачиваем

In [6]:
url = 'https://наш.дом.рф/сервисы/каталог-новостроек/объект'


def download(ids, cnt, lock, force=False):
    with webdriver.Chrome(options=chrome_options) as browser:

        for id in ids:
            dir = os.path.join(SRC, str(int(id) // 1000 * 1000))
            file = os.path.join(dir, f'{id}.html')

            if not force and os.path.exists(file):
                with lock:
                    cnt.value += 1
                
                continue
        
            browser.get(f'{url}/{id}')

            html = browser.page_source

            os.makedirs(dir, exist_ok=True)
            
            with open(file, 'w') as f:
                f.write(html)
            
            with lock:
                cnt.value += 1


def multiprosses_download(n_proc=2, force=False):
    manager = Manager()
    cnt = manager.Value('i', 0)
    lock = Lock()

    ids_list = list(ids)
    batch_sz = (len(ids_list) + N_PROC - 1) // N_PROC
    batches = [ids_list[i * batch_sz:(i + 1) * batch_sz] for i in range(N_PROC)]

    with tqdm(total=len(ids_list)) as progress_bar:
            processes = [Process(target=download, args=(batch, cnt, lock, force)) for batch in batches]

            for process in processes:
                process.start()
            
            while any(process.is_alive() for process in processes):
                progress_bar.update(n=cnt.value - progress_bar.n)

            for process in processes:
                process.join()


In [7]:
N_PROC = 8

multiprosses_download(N_PROC, force=False)

  0%|          | 0/1000 [00:00<?, ?it/s]